In [1]:
from pyspark.sql.functions import udf, lit
from math import radians, cos, sin, asin, sqrt
from pyspark.sql.types import FloatType
from pyspark.sql.types import *
import pyspark 
from delta import *
from delta.tables import DeltaTable

def main():
    
    # Calculate the haversine distance between two sets of latitudes and longitudes
    @udf(returnType=FloatType())
    def haversine_distance(lat1, lon1, lat2, lon2) -> float:
        R = 6371  # radius of the earth in kilometers
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        distance = R * c
        return distance
    
    # Constants
    LATITUDE = 40.7128
    LONGITUDE = -74.0060
    TABLE_PATH = "hdfs:///project/data/business_data/delta_table"
    CSV_PATH = "hdfs:///project/data/business_data/yelp_academic_dataset_business_64x.csv"
    
    # Initiate spark
    builder = pyspark.sql.SparkSession.builder.appName("spark_distance") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.executor.memory", "4g") \
        .config("spark.executor.cores", 4) \
        .config("spark.submit.deployMode", "client")
    spark = configure_spark_with_delta_pip(builder).getOrCreate()

    # Create dataframe from csv file
    schema = "Business_id STRING, Latitude FLOAT, Longitude FLOAT, Stars FLOAT"
    df = spark.read.csv(CSV_PATH, schema=schema)

    # Calculate haversine distances
    df = df.withColumn("Distance", haversine_distance(df.Latitude, df.Longitude, lit(LATITUDE), lit(LONGITUDE)))

    # If no delta table exists, save and exit
    if not DeltaTable.isDeltaTable(spark, TABLE_PATH):
        df.write.format("delta").save(TABLE_PATH)
        return
        
    # Upsert delta table
    delta_table = DeltaTable.forPath(spark, TABLE_PATH)
    delta_table.alias("old") \
        .merge(
            df.alias("new"),
            "old.Business_id = new.Business_id"
        ) \
        .whenMatchedUpdate(set=
            {
                "Business_id": "new.Business_id",
                "Latitude": "new.Latitude",
                "Longitude": "new.Longitude",
                "Stars": "new.Stars",
                "Distance": "new.Distance"
            }
        ) \
        .whenNotMatchedInsert(values=
            {
                "Business_id": "new.Business_id",
                "Latitude": "new.Latitude",
                "Longitude": "new.Longitude",
                "Stars": "new.Stars",
                "Distance": "new.Distance"
            }
        ) \
        .execute()


if __name__ == "__main__":
    main()

:: loading settings :: url = jar:file:/home/ubuntu/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-801de134-db9a-4d67-8e1b-4607d9277e6f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 222ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   

23/04/24 19:36:54 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
23/04/24 19:37:09 WARN Client: Same path resource file:///home/ubuntu/.ivy2/jars/io.delta_delta-core_2.12-2.3.0.jar added multiple times to distributed cache.
23/04/24 19:37:09 WARN Client: Same path resource file:///home/ubuntu/.ivy2/jars/io.delta_delta-storage-2.3.0.jar added multiple times to distributed cache.
23/04/24 19:37:09 WARN Client: Same path resource file:///home/ubuntu/.ivy2/jars/org.antlr_antlr4-runtime-4.8.jar added multiple times to distributed cache.


23/04/24 19:37:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
